# How to predict the best business to open in Rome

### Paolo Mammoliti

August 13, 2020


Importing All the packages and libraries needed for this project

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

I'll now use a wikipedia page to get the Roma's borough data located in a table

In [5]:
source = requests.get("https://it.wikipedia.org/wiki/Municipi_di_Roma").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["Borough","Population","Area","Density","President"])
df.head()

,Borough,Population,Area,Density,President
0,I Centro Storico\n,170 328\n,"20,09\n","8 478,25\n",Sabrina Alfonsi (PD)\n
1,II Parioli/Nomentano\n,168 410\n,"19,66\n","8 566,12\n",Francesca Del Bello (PD)\n
2,III Monte Sacro\n,205 832\n,"98,03\n","2 099,68\n",Giovanni Caudo (centrosinistra)\n
3,IV Tiburtino\n,175 921\n,"48,94\n","3 594,63\n",Virginia Raggi (Commissario Straordinario)\n
4,V Prenestino/Centocelle\n,245 073\n,"26,92\n","9 103,75\n",Giovanni Boccuzzi (M5S)\n


Replacing some weirds characters with the empty ones

In [6]:
df["Population"] = df["Population"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Area"] = df["Area"].str.replace("\n","")
df["Density"] = df["Density"].str.replace("\n","")
df["President"] = df["President"].str.replace("\n","")
df.head()

,Borough,Population,Area,Density,President
0,I Centro Storico,170 328,"20,09","8 478,25",Sabrina Alfonsi (PD)
1,II Parioli/Nomentano,168 410,"19,66","8 566,12",Francesca Del Bello (PD)
2,III Monte Sacro,205 832,"98,03","2 099,68",Giovanni Caudo (centrosinistra)
3,IV Tiburtino,175 921,"48,94","3 594,63",Virginia Raggi (Commissario Straordinario)
4,V Prenestino/Centocelle,245 073,"26,92","9 103,75",Giovanni Boccuzzi (M5S)


Dropping a no needed data frame column

In [7]:
df.drop('President', axis=1, inplace=True)

Dropping the last two rows of the data frame with some info not needed

In [8]:
df.drop([15, 16])

,Borough,Population,Area,Density
0,I Centro Storico,170 328,"20,09","8 478,25"
1,II Parioli/Nomentano,168 410,"19,66","8 566,12"
2,III Monte Sacro,205 832,"98,03","2 099,68"
3,IV Tiburtino,175 921,"48,94","3 594,63"
4,V Prenestino/Centocelle,245 073,"26,92","9 103,75"
5,VI Roma delle Torri,257 556,"113,88","2 261,64"
6,VII Appio-Latino/Tuscolana/Cinecittà,307 184,"45,84","6 701,22"
7,VIII Appia Antica,130 784,"47,15","2 773,79"
8,IX Eur,183 343,"183,31","1 000,18"
9,X Ostia/Acilia,231 701,"150,74","1 537,09"


Now we'll load and import a csv file with the Roma's boroughs latitude and longitude data. I'll remove the sentitive credentials to share the notebook

In [11]:

df_data_3 = pd.read_csv('/content/datiRoma.csv')

In [13]:
df_data_3

,Borough,Latitude,Longitude
0,I Centro Storico,41.893056,12.482778
1,II Parioli/Nomentano,41.929958,12.518931
2,III Monte Sacro,41.936080,12.535116
3,IV Tiburtino,41.933491,12.598746
4,V Prenestino/Centocelle,41.890665,12.548488
5,VI Roma delle Torri,41.869657,12.632731
6,VII Appio-Latino/Tuscolana/Cinecitta,41.881700,12.522800
7,VIII Appia Antica,41.841228,12.484290
8,IX Eur,41.814879,12.479980
9,X Ostia/Acilia,41.730660,12.280531


I'll now merge the two dataframe created above

In [14]:
df_Roma = pd.merge(df, df_data_3, how='right', left_on = 'Borough', right_on = 'Borough')
# remove the "Postal Code" column
df_Roma.head()

,Borough,Population,Area,Density,Latitude,Longitude
0,I Centro Storico,170 328,"20,09","8 478,25",41.893056,12.482778
1,II Parioli/Nomentano,168 410,"19,66","8 566,12",41.929958,12.518931
2,III Monte Sacro,205 832,"98,03","2 099,68",41.936080,12.535116
3,IV Tiburtino,175 921,"48,94","3 594,63",41.933491,12.598746
4,V Prenestino/Centocelle,245 073,"26,92","9 103,75",41.890665,12.548488


In [15]:
df_Roma

,Borough,Population,Area,Density,Latitude,Longitude
0,I Centro Storico,170 328,"20,09","8 478,25",41.893056,12.482778
1,II Parioli/Nomentano,168 410,"19,66","8 566,12",41.929958,12.518931
2,III Monte Sacro,205 832,"98,03","2 099,68",41.936080,12.535116
3,IV Tiburtino,175 921,"48,94","3 594,63",41.933491,12.598746
4,V Prenestino/Centocelle,245 073,"26,92","9 103,75",41.890665,12.548488
5,VI Roma delle Torri,257 556,"113,88","2 261,64",41.869657,12.632731
6,VIII Appia Antica,130 784,"47,15","2 773,79",41.841228,12.484290
7,IX Eur,183 343,"183,31","1 000,18",41.814879,12.479980
8,X Ostia/Acilia,231 701,"150,74","1 537,09",41.730660,12.280531
9,XI Arvalia/Portuense,155 652,"71,48","2 177,56",41.855282,12.444762


Removing 3 more columns

In [16]:
df_Roma.drop(["Population","Area","Density"], axis=1, inplace=True)

Using the geolocator library to find the geocrapical coordinate of Rome

In [17]:
address = "Rome, IT"

geolocator = Nominatim(user_agent="roma_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rome are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rome are 41.8933203, 12.4829321.


Creating the Roma map with Folium

In [18]:
# create map of Rome using latitude and longitude values
map_Roma = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Roma

Adding the boroughs circle marker with Folium using the data frame created

In [19]:
for lat, lng, borough in zip(
        df_Roma['Latitude'], 
        df_Roma['Longitude'], 
        df_Roma['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Roma)  

map_Roma

Adding my Foursquare API credentials to find the venues (this will be removed to share the notebook)

In [20]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

In [21]:
borough_name = df_Roma.loc[0, 'Borough']
print(f"The first neighborhood's name is '{borough_name}'.")

The first neighborhood's name is 'I Centro Storico'.


In [22]:
borough_latitude = df_Roma.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = df_Roma.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of I Centro Storico are 41.893056, 12.482778.


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Finding a venues list situated in the first borough "I Centro Storico"

In [25]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Piazza del Campidoglio,Plaza,41.893321,12.482956
1,Terrazza delle Quadrighe,Scenic Lookout,41.894346,12.483336
2,Foro di Cesare,Historic Site,41.894128,12.485232
3,Musei Capitolini,Art Museum,41.893343,12.482885
4,Capitoline Hill (Campidoglio),Scenic Lookout,41.893462,12.483588
...,...,...,...,...
66,Curia (Curia Julia),Historic Site,41.892938,12.485391
67,Chiesa Santa Maria Antiqua,Historic Site,41.891437,12.485867
68,Lacus Iuturnae (Fonte di Giuturna),Fountain,41.891406,12.485905
69,Domus Tiberiana,Historic Site,41.890202,12.486069


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
df_Roma_venues = getNearbyVenues(names=df_Roma['Borough'],
                                   latitudes=df_Roma['Latitude'],
                                   longitudes=df_Roma['Longitude']
                                  )

In [28]:
df_Roma_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,I Centro Storico,41.893056,12.482778,Piazza del Campidoglio,41.893321,12.482956,Plaza
1,I Centro Storico,41.893056,12.482778,Terrazza delle Quadrighe,41.894346,12.483336,Scenic Lookout
2,I Centro Storico,41.893056,12.482778,Foro di Cesare,41.894128,12.485232,Historic Site
3,I Centro Storico,41.893056,12.482778,Musei Capitolini,41.893343,12.482885,Art Museum
4,I Centro Storico,41.893056,12.482778,Capitoline Hill (Campidoglio),41.893462,12.483588,Scenic Lookout


Calculating the number of venue's categories on each borough

In [29]:
df_Roma_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
I Centro Storico,71,71,71,71,71,71
II Parioli/Nomentano,33,33,33,33,33,33
III Monte Sacro,26,26,26,26,26,26
IV Tiburtino,8,8,8,8,8,8
IX Eur,6,6,6,6,6,6
V Prenestino/Centocelle,10,10,10,10,10,10
VI Roma delle Torri,7,7,7,7,7,7
VII Appio-Latino/Tuscolana/Cinecitta,13,13,13,13,13,13
VIII Appia Antica,12,12,12,12,12,12


In [30]:
print('There are {} uniques categories.'.format(len(df_Roma_venues['Venue Category'].unique())))

There are 84 uniques categories.


In [31]:
# one hot encoding
df_Roma_onehot = pd.get_dummies(df_Roma_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Roma_onehot['Borough'] = df_Roma_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [df_Roma_onehot.columns[-1]] + list(df_Roma_onehot.columns[:-1])
df_Roma_onehot = df_Roma_onehot[fixed_columns]

df_Roma_onehot.head()

,Borough,African Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Basketball Court,Beach,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Convenience Store,Convention Center,Department Store,Dessert Shop,Falafel Restaurant,Fast Food Restaurant,Fish Market,Food,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Gym,Gym / Fitness Center,...,Ice Cream Shop,Irish Pub,Italian Restaurant,Japanese Restaurant,Laser Tag,Light Rail Station,Market,Mexican Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Museum,Noodle House,Park,Pastry Shop,Photography Lab,Piadineria,Pizza Place,Plaza,Pub,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Spanish Restaurant,Steakhouse,Supermarket,Temple,Thai Restaurant,Theater,Tourist Information Center,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar
0,I Centro Storico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,I Centro Storico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,I Centro Storico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,I Centro Storico,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,I Centro Storico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Calculatering the percentage of the venues categories on each borough

In [32]:
df_Roma_grouped = df_Roma_onehot.groupby('Borough').mean().reset_index()
df_Roma_grouped.head()

,Borough,African Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Basketball Court,Beach,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Convenience Store,Convention Center,Department Store,Dessert Shop,Falafel Restaurant,Fast Food Restaurant,Fish Market,Food,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Gym,Gym / Fitness Center,...,Ice Cream Shop,Irish Pub,Italian Restaurant,Japanese Restaurant,Laser Tag,Light Rail Station,Market,Mexican Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Museum,Noodle House,Park,Pastry Shop,Photography Lab,Piadineria,Pizza Place,Plaza,Pub,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Spanish Restaurant,Steakhouse,Supermarket,Temple,Thai Restaurant,Theater,Tourist Information Center,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar
0,I Centro Storico,0.0,0.000,0.014085,0.0,0.000000,0.028169,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.014085,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014085,0.000,0.0,0.014085,0.028169,0.000000,0.000000,0.028169,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.014085,0.042254,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.056338,0.000,0.028169,0.0,0.000000,0.000000,0.0,0.000000,0.014085,0.028169,0.042254,0.014085,0.014085,0.014085,0.0,0.014085,0.042254,0.0,0.000,0.0,0.0,0.000000,0.000000,0.084507,0.0,0.014085,0.014085,0.028169,0.000000,0.0,0.0
1,II Parioli/Nomentano,0.0,0.000,0.000000,0.0,0.030303,0.000000,0.0,0.0,0.030303,0.000000,0.060606,0.000000,0.0,0.0,0.0,0.0,0.060606,0.0,0.000000,0.030303,0.000000,0.0,0.0,0.0,0.090909,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.121212,0.000000,0.060606,0.030303,0.030303,0.0,0.030303,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.030303,0.060606,0.0,0.030303,0.030303,0.090909,0.030303,0.030303,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.030303,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,III Monte Sacro,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.038462,0.038462,0.000000,0.038462,0.0,0.0,0.0,0.0,0.076923,0.0,0.000000,0.000000,0.115385,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.038462,0.000000,0.000000,0.038462,0.0,0.0,0.0,0.0,0.0,...,0.038462,0.000000,0.038462,0.038462,0.000000,0.0,0.000000,0.038462,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.115385,0.038462,0.076923,0.000000,0.000000,0.000000,0.0,0.038462,0.000000,0.0,0.000,0.0,0.0,0.038462,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.038462,0.0,0.0
3,IV Tiburtino,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.125,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.125,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.125,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,IX Eur,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


Discovering the most commons venue's categories on each borough

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = df_Roma_grouped['Borough']

for ind in np.arange(df_Roma_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_Roma_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,I Centro Storico,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,Bar,Plaza,Garden
1,II Parioli/Nomentano,Ice Cream Shop,Plaza,Dessert Shop,Bookstore,Café,Italian Restaurant,Pastry Shop,Park,Pub,Restaurant
2,III Monte Sacro,Cocktail Bar,Pizza Place,Hotel,Café,Pub,Mexican Restaurant,Historic Site,French Restaurant,Steakhouse,Plaza
3,IV Tiburtino,Café,American Restaurant,Hotel,Fast Food Restaurant,Italian Restaurant,Motorcycle Shop,Shoe Store,Wine Bar,Fountain,Dessert Shop
4,IX Eur,Hotel,Pizza Place,Restaurant,Dessert Shop,Wine Bar,Food,Convenience Store,Convention Center,Department Store,Falafel Restaurant
5,V Prenestino/Centocelle,Gym,Supermarket,Italian Restaurant,Market,Noodle House,Café,Sandwich Place,African Restaurant,Pizza Place,Asian Restaurant
6,VI Roma delle Torri,Pizza Place,Plaza,Theater,Hotel,Supermarket,Bus Station,Shopping Mall,Fish Market,Convenience Store,Convention Center
7,VII Appio-Latino/Tuscolana/Cinecitta,Italian Restaurant,Trattoria/Osteria,Hotel,Photography Lab,Asian Restaurant,Thai Restaurant,Pizza Place,Hostel,Plaza,Bistro
8,VIII Appia Antica,Café,Italian Restaurant,Plaza,Japanese Restaurant,Bistro,Salad Place,Mexican Restaurant,Wine Bar,Food,Department Store
9,X Ostia/Acilia,Pizza Place,Italian Restaurant,Seafood Restaurant,Beach,Ice Cream Shop,Café,Cocktail Bar,Restaurant,Fast Food Restaurant,Hotel


Creating 5 random clusters showing the closest boroughs based on the venues discriminance

In [34]:
# set number of clusters
kclusters = 5

df_Roma_grouped_clustering = df_Roma_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_Roma_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 2, 1, 3, 3, 3, 2, 3], dtype=int32)

In [35]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_Roma_merged = df_Roma

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_Roma_merged = df_Roma_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

df_Roma_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,I Centro Storico,41.893056,12.482778,3,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,Bar,Plaza,Garden
1,II Parioli/Nomentano,41.929958,12.518931,0,Ice Cream Shop,Plaza,Dessert Shop,Bookstore,Café,Italian Restaurant,Pastry Shop,Park,Pub,Restaurant
2,III Monte Sacro,41.936080,12.535116,3,Cocktail Bar,Pizza Place,Hotel,Café,Pub,Mexican Restaurant,Historic Site,French Restaurant,Steakhouse,Plaza
3,IV Tiburtino,41.933491,12.598746,2,Café,American Restaurant,Hotel,Fast Food Restaurant,Italian Restaurant,Motorcycle Shop,Shoe Store,Wine Bar,Fountain,Dessert Shop
4,V Prenestino/Centocelle,41.890665,12.548488,3,Gym,Supermarket,Italian Restaurant,Market,Noodle House,Café,Sandwich Place,African Restaurant,Pizza Place,Asian Restaurant


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        df_Roma_merged['Latitude'], 
        df_Roma_merged['Longitude'], 
        df_Roma_merged['Borough'], 
        df_Roma_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
df_Roma_merged.loc[df_Roma_merged['Cluster Labels'] == 0, df_Roma_merged.columns[[0] + list(range(3, df_Roma_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,II Parioli/Nomentano,0,Ice Cream Shop,Plaza,Dessert Shop,Bookstore,Café,Italian Restaurant,Pastry Shop,Park,Pub,Restaurant
13,XV Milvio,0,Bookstore,Trattoria/Osteria,Café,Restaurant,Basketball Court,Brewery,Sandwich Place,Light Rail Station,Fountain,Dessert Shop


In [38]:
df_Roma_merged.loc[df_Roma_merged['Cluster Labels'] == 1, df_Roma_merged.columns[[0] + list(range(3, df_Roma_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,IX Eur,1,Hotel,Pizza Place,Restaurant,Dessert Shop,Wine Bar,Food,Convenience Store,Convention Center,Department Store,Falafel Restaurant


In [39]:
df_Roma_merged.loc[df_Roma_merged['Cluster Labels'] == 2, df_Roma_merged.columns[[0] + list(range(3, df_Roma_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,IV Tiburtino,2,Café,American Restaurant,Hotel,Fast Food Restaurant,Italian Restaurant,Motorcycle Shop,Shoe Store,Wine Bar,Fountain,Dessert Shop
6,VIII Appia Antica,2,Café,Italian Restaurant,Plaza,Japanese Restaurant,Bistro,Salad Place,Mexican Restaurant,Wine Bar,Food,Department Store


In [40]:
df_Roma_merged.loc[df_Roma_merged['Cluster Labels'] == 3, df_Roma_merged.columns[[0] + list(range(3, df_Roma_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,I Centro Storico,3,Historic Site,Hotel,Temple,Monument / Landmark,Italian Restaurant,Pub,Scenic Lookout,Bar,Plaza,Garden
2,III Monte Sacro,3,Cocktail Bar,Pizza Place,Hotel,Café,Pub,Mexican Restaurant,Historic Site,French Restaurant,Steakhouse,Plaza
4,V Prenestino/Centocelle,3,Gym,Supermarket,Italian Restaurant,Market,Noodle House,Café,Sandwich Place,African Restaurant,Pizza Place,Asian Restaurant
5,VI Roma delle Torri,3,Pizza Place,Plaza,Theater,Hotel,Supermarket,Bus Station,Shopping Mall,Fish Market,Convenience Store,Convention Center
8,X Ostia/Acilia,3,Pizza Place,Italian Restaurant,Seafood Restaurant,Beach,Ice Cream Shop,Café,Cocktail Bar,Restaurant,Fast Food Restaurant,Hotel
9,XI Arvalia/Portuense,3,Pizza Place,Supermarket,Café,Gym / Fitness Center,Gym,Clothing Store,Plaza,Bistro,Italian Restaurant,Fast Food Restaurant
10,XII Monte Verde,3,Pizza Place,Café,Restaurant,Ice Cream Shop,Italian Restaurant,Gym / Fitness Center,Garden,Food,Market,Plaza
11,XIII Aurelio,3,Hotel,Café,Italian Restaurant,Plaza,Supermarket,Steakhouse,Gym / Fitness Center,Fast Food Restaurant,Clothing Store,Department Store
14,VII Appio-Latino/Tuscolana/Cinecitta,3,Italian Restaurant,Trattoria/Osteria,Hotel,Photography Lab,Asian Restaurant,Thai Restaurant,Pizza Place,Hostel,Plaza,Bistro


In [41]:
df_Roma_merged.loc[df_Roma_merged['Cluster Labels'] == 4, df_Roma_merged.columns[[0] + list(range(3, df_Roma_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,XIV Monte Mario,4,Plaza,Park,Chinese Restaurant,Italian Restaurant,Supermarket,Wine Bar,Food,Department Store,Dessert Shop,Falafel Restaurant
